In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, StackingClassifier

# Read the data
df_train = pd.read_csv('preprocessed_train.csv', index_col='TX_ID')
df_test = pd.read_csv('preprocessed_test.csv')

# Keep a copy of TX_ID in X_test for saving predictions later
X_test_ids = df_test['TX_ID']

# Remove rows with missing target, separate target from predictors
df_train.dropna(axis=0, subset=['TX_FRAUD'], inplace=True)
y = df_train.TX_FRAUD
df_train.drop(['TX_FRAUD'], axis=1, inplace=True)

# Columns to drop that cannot be encoded or contain missing values
drop_cols = [
  'CUSTOMER_ID',
  'TERMINAL_ID',
  'CARD_DATA',
  'CARD_EXPIRY_DATE',
  'FAILURE_CODE',
  'FAILURE_REASON',
  'ACQUIRER_ID',
  'CARDHOLDER_AUTH_METHOD',
  'MERCHANT_ID'
]

# Drop the unwanted columns from both train and test sets
df_train.drop(drop_cols, axis=1, inplace=True)
df_test.drop(drop_cols, axis=1, inplace=True)

# Split the training data into train and validation sets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(df_train, y, train_size=0.8, test_size=0.2, random_state=0)

# List of numeric and object columns
numeric_cols = [
  'TX_AMOUNT',
  'TRANSACTION_GOODS_AND_SERVICES_AMOUNT',
  'TRANSACTION_CASHBACK_AMOUNT',
]

object_cols = [ # Apply one-shot encoding
  'CARD_BRAND',
  'TRANSACTION_TYPE',
  'TRANSACTION_STATUS',
  'TRANSACTION_CURRENCY',
  'CARD_COUNTRY_CODE',
  'IS_RECURRING_TRANSACTION'
]

# Combine numeric and categorical columns into a single list for further processing
my_cols = numeric_cols + object_cols

# Select the relevant columns for train, validation, and test sets
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()
X_test = df_test[my_cols].copy()

# One-hot encode categorical columns only (using pandas get_dummies)
X_train = pd.get_dummies(X_train, columns=object_cols)
X_valid = pd.get_dummies(X_valid, columns=object_cols)
X_test = pd.get_dummies(X_test, columns=object_cols)

# Align the columns of training, validation, and test sets
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

# Fill missing columns with 0 (after alignment)
X_valid = X_valid.fillna(0)
X_test = X_test.fillna(0)

# Re-attach TX_ID to X_test for later use in predictions
X_test['TX_ID'] = X_test_ids

# Check column consistency
print(X_test.columns)
print()
print(X_valid.columns)


Index(['TX_AMOUNT', 'TRANSACTION_GOODS_AND_SERVICES_AMOUNT',
       'TRANSACTION_CASHBACK_AMOUNT', 'CARD_BRAND_AMEX', 'CARD_BRAND_Discover',
       'CARD_BRAND_MasterCard', 'CARD_BRAND_Visa',
       'TRANSACTION_TYPE_Cash Advance/Withdrawal', 'TRANSACTION_TYPE_Purchase',
       'TRANSACTION_TYPE_Purchase with cashback', 'TRANSACTION_TYPE_Refund',
       'TRANSACTION_STATUS_Authorized', 'TRANSACTION_STATUS_Captured',
       'TRANSACTION_STATUS_Rejected', 'TRANSACTION_STATUS_Settled',
       'TRANSACTION_CURRENCY_AED', 'TRANSACTION_CURRENCY_CAD',
       'TRANSACTION_CURRENCY_CHF', 'TRANSACTION_CURRENCY_CNY',
       'TRANSACTION_CURRENCY_EUR', 'TRANSACTION_CURRENCY_GBP',
       'TRANSACTION_CURRENCY_HKD', 'TRANSACTION_CURRENCY_JPY',
       'TRANSACTION_CURRENCY_MDL', 'TRANSACTION_CURRENCY_RON',
       'TRANSACTION_CURRENCY_USD', 'CARD_COUNTRY_CODE_AE',
       'CARD_COUNTRY_CODE_BE', 'CARD_COUNTRY_CODE_BT', 'CARD_COUNTRY_CODE_CA',
       'CARD_COUNTRY_CODE_DE', 'CARD_COUNTRY_CODE_DK', 'CAR

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

model_r2 = RandomForestRegressor(
    n_estimators=300,
    max_depth=20,
    min_samples_split=5,
    min_samples_leaf=2,
    max_features='sqrt',
    bootstrap=True,
    oob_score=True,
    n_jobs=-2,
    random_state=0
)

model_r2.fit(X_train, y_train)
preds2 = model_r2.predict(X_valid)

mae = mean_absolute_error(y_valid, preds2)
roc_auc = roc_auc_score(y_valid, preds2)

print(f"Mean Absolute Error: {mae:.4f}")
print(f"ROC AUC: {roc_auc:.4f}")


Mean Absolute Error: 0.0473
ROC AUC: 0.5669


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'max_features': ['sqrt', 'log2'],
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)


In [ ]:
grid_search = GridSearchCV(RandomForestRegressor(), param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

In [18]:
# Drop the 'TX_ID' column from X_test before prediction
test_preds = model_r2.predict(X_test.drop('TX_ID', axis=1))

# Create a submission DataFrame with 'TX_ID' and the predicted 'TX_FRAUD'
output = pd.DataFrame({
    'TX_ID': X_test.TX_ID,  # Retain TX_ID for the output
    'TX_FRAUD': test_preds
})

# Save to CSV, ensuring you don't set the index to 'TX_ID'
output.to_csv('submission.csv', index=False)
